In [37]:
import numpy as np 
import pandas as pd

In [38]:
train = pd.read_csv('../data/merged_train.csv')
train_alt = pd.read_csv('../data/merged_train_alt.csv')
test = pd.read_csv('../data/merged_test.csv')
sample_subm = pd.read_csv('../data/sample_submission.csv')

/var/folders/26/840b8l353yv58xh3z45hs1tc0000gn/T/ipykernel_5658/97445140.py:1: DtypeWarning: Columns (14,16,18,20) have mixed types. Specify dtype option on import or set low_memory=False.
  train = pd.read_csv('../data/merged_train.csv')
/var/folders/26/840b8l353yv58xh3z45hs1tc0000gn/T/ipykernel_5658/97445140.py:2: DtypeWarning: Columns (14,16,18,20) have mixed types. Specify dtype option on import or set low_memory=False.
  train_alt = pd.read_csv('../data/merged_train_alt.csv')


## Baseline Model I: Weighted Mean Sales

|  Variable  | Description 
|:--------|:--------|
$D$         |   day of the week (Monday-Sunday)
$W$         |   week of the year
$S$         |   Favorita store number (1-54)
$F$         |   product family, e.g. baby care products
$Y$         |   year  (2013, 2014, 2015, 2016)
$\textnormal{Sales}_{D,W,S,F}(Y)$    |   total sales of $F$ products on day $D$, week $W$, store $S$, in the year $Y$.

Our baseline model predicts $\textnormal{Sales}_{D,S,F}(2017)$ using a weighted average of previous years sales.

More recent years are more heavily weighted.

$$\textnormal{Sales}_{D,W,S,F}(2017) = \sum_{Y=2013}^{2016}\lambda_{Y}\cdot\textnormal{Sales}_{D,W,S,F}(Y) \hspace{0.5in} \textnormal{where }\lambda_{Y}=0.1+0.1(Y-2013)$$


In [ ]:
years = [2013, 2014, 2015, 2016]

for year in years:
    # Splits training set by year
    # Renames 'sales' column to f'sales_{year}'
    df_year = train[train['year'] == year].copy()
    df_year = df_year.rename(columns={'sales': f'sales_{year}'})
    
    # Merge the previous year sales to test data
    # Merge along day of the week, week number, store number, and product family
    test = test.merge( df_year[['week_number', 'day_of_week', 'store_nbr', 'family', f'sales_{year}']], 
                            on=['week_number', 'day_of_week', 'store_nbr', 'family'], 
                            how='left'    
    )

test['mean_sales'] = 0.1*test['sales_2013']+ 0.2*test['sales_2014']+ 0.3*test['sales_2015']+0.4*test['sales_2016']

In [ ]:
def baseline1_submission():
    sample_subm['sales'] = test['mean_sales']
    #sample_subm.to_csv('/kaggle/working/submission.csv',index=False)
    # it scores 0.90242

## Baseline Model II: Rolling Average

In [ ]:
# Reset the testing set
test = pd.read_csv('../data/merged_test.csv')

# Combine training and testing sets to compute rolling averages
df_whole = pd.concat([train, test], ignore_index=True)

# Calculate the rolling average
avg_sales=df_whole.groupby(['store_nbr', 'family'])['sales'].transform(lambda x: x.rolling(window=30, min_periods=1).mean())

# Add rolling averages as a columm
df_whole['avg_sales']=avg_sales

# Select the testing set dates
df_whole[len(train):]

In [ ]:
def baseline2_submission():
    sample_subm['sales'] = df_whole['avg_sales'].iloc[len(train):].reset_index(drop=True)
    #sample_subm.to_csv('/kaggle/working/submission.csv',index=False)
    # it scores 0.46141 